#### Toronto Neibourghood 

### Capstone Project Assignmet

#### Importing the necessary library 

In [4]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

import json # library to handle JSON files
!conda install -c conda-forge geopy --yes 

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
!conda install -c conda-forge bs4 --yes
!conda install -c conda-forge/label/cf202003 bs4 --yes
import requests # library to handle requests
from bs4 import BeautifulSoup # library to parse HTML and XML documents


from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print("Libraries imported.")

Solving environment: done

# All requested packages already installed.

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - bs4


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    beautifulsoup4-4.9.1       |   py36h9f0ad1d_0         163 KB  conda-forge
    bs4-4.9.1                  |                0           4 KB  conda-forge
    soupsieve-2.0.1            |   py36h9f0ad1d_0          56 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         223 KB

The following NEW packages will be INSTALLED:

  beautifulsoup4     conda-forge/linux-64::beautifulsoup4-4.9.1-py36h9f0ad1d_0
  bs4                conda-forge/noarch::bs4-4.9.1-0
  soupsieve          conda-forge/linux-64::soupsieve-2.0.1-py36h9f0ad1d_0



beautifulsoup4-4.9.

#### Scraping data from Wikipedia page into a DataFrame

In [6]:
data = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(data, 'html.parser')

In [8]:
# create three lists to store table data
PostalCodeL = []
BoroughL = []
NeighborhoodL= []

In [18]:
# append the data into the respective lists
for row in soup.find('table').find_all('tr'):
    cells = row.find_all('td')
    if(len(cells) > 0):
        PostalCodeL.append(cells[0].text)
        BoroughL.append(cells[1].text)
        NeighborhoodL.append(cells[2].text) # avoid new lines in neighborhood cell .rstrip('\n')

In [19]:
# create a new DataFrame from the three list
Toronto_df = pd.DataFrame({"PostalCode": PostalCodeL,
                           "Borough": BoroughL,
                           "Neighborhood": NeighborhoodL})

Toronto_df.head()

,PostalCode,Borough,Neighborhood
0,M1A\n,Not assigned\n,Not assigned
1,M2A\n,Not assigned\n,Not assigned
2,M3A\n,North York\n,Parkwoods
3,M4A\n,North York\n,Victoria Village
4,M5A\n,Downtown Toronto\n,"Regent Park, Harbourfront"


In [21]:
# drop cells with a borough that is Not assigned/n
Toronto_df_dropna = Toronto_df[Toronto_df.Borough != "Not assigned\n"].reset_index(drop=True)
Toronto_df_dropna.head()

,PostalCode,Borough,Neighborhood
0,M3A\n,North York\n,Parkwoods
1,M4A\n,North York\n,Victoria Village
2,M5A\n,Downtown Toronto\n,"Regent Park, Harbourfront"
3,M6A\n,North York\n,"Lawrence Manor, Lawrence Heights"
4,M7A\n,Downtown Toronto\n,"Queen's Park, Ontario Provincial Government"


In [24]:
# drop cells with a borough that is Not assigned
Toronto_df_dropna1 = Toronto_df_dropna[Toronto_df_dropna.Borough != "Not assigned"].reset_index(drop=True)
Toronto_df_dropna1.head()

,PostalCode,Borough,Neighborhood
0,M3A\n,North York\n,Parkwoods
1,M4A\n,North York\n,Victoria Village
2,M5A\n,Downtown Toronto\n,"Regent Park, Harbourfront"
3,M6A\n,North York\n,"Lawrence Manor, Lawrence Heights"
4,M7A\n,Downtown Toronto\n,"Queen's Park, Ontario Provincial Government"


In [25]:
# group neighborhoods in the same borough
Toronto_df_grouped = Toronto_df_dropna1.groupby(["PostalCode", "Borough"], as_index=False).agg(lambda x: ", ".join(x))
Toronto_df_grouped.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1B\n,Scarborough\n,"Malvern, Rouge, Malvern, Rouge\n, Malvern, Rou..."
2,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
3,M1C\n,Scarborough\n,"Rouge Hill, Port Union, Highland Creek, Rouge ..."
4,M1E,Scarborough,"Guildwood, Morningside, West Hill"


In [26]:

# for Neighborhood="Not assigned", make the value the same as Borough
for index, row in Toronto_df_grouped.iterrows():
    if row["Neighborhood"] == "Not assigned":
        row["Neighborhood"] = row["Borough"]
        
Toronto_df_grouped.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1B\n,Scarborough\n,"Malvern, Rouge, Malvern, Rouge\n, Malvern, Rou..."
2,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
3,M1C\n,Scarborough\n,"Rouge Hill, Port Union, Highland Creek, Rouge ..."
4,M1E,Scarborough,"Guildwood, Morningside, West Hill"


In [27]:

# create a new test dataframe
column_names = ["PostalCode", "Borough", "Neighborhood"]
test_df = pd.DataFrame(columns=column_names)

test_list = ["M5G", "M2H", "M4B", "M1J", "M4G", "M4M", "M1R", "M9V", "M9L", "M5V", "M1B", "M5A"]

for postcode in test_list:
    test_df = test_df.append(Toronto_df_grouped[Toronto_df_grouped["PostalCode"]==postcode], ignore_index=True)
    
test_df

,PostalCode,Borough,Neighborhood
0,M5G,Downtown Toronto,Central Bay Street
1,M2H,North York,Hillcrest Village
2,M4B,East York,"Parkview Hill, Woodbine Gardens"
3,M1J,Scarborough,Scarborough Village
4,M4G,East York,Leaside
5,M4M,East Toronto,Studio District
6,M1R,Scarborough,"Wexford, Maryvale"
7,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest..."
8,M9L,North York,Humber Summit
9,M5V,Downtown Toronto,"CN Tower, King and Spadina, Railway Lands, Har..."


In [29]:

# print the number of rows of the cleaned dataToronto_df_grouped.head()
Toronto_df_grouped.shape


(206, 3)